In [7]:
import numpy as np
import pydrake
from pydrake.solvers import ik
from pydrake.examples.pendulum import PendulumWithBotVis

In [8]:
r = pydrake.rbm.RigidBodyManipulator("../examples/Pendulum/Pendulum.urdf")

In [9]:
q = -0.9
posture_constraint = ik.PostureConstraint(r)
posture_constraint.setJointLimits(np.array([[6]], dtype=np.int32), 
                                  np.array([[q]]), 
                                  np.array([[q]]))

In [10]:
lc = pydrake.wrappers.lcm.LCM()
p = PendulumWithBotVis(lc)

In [6]:
q_seed = np.vstack((np.zeros((6,1)), 
                    p.getRandomState()[:1]))
q_nom = np.vstack((np.zeros((6,1)), 
                   0.))
options = ik.IKoptions(r)
results = pydrake.solvers.ik.inverseKinSimple(r, 
                              q_seed, 
                              q_nom, 
                              [posture_constraint], 
                              options)

NameError: name 'ik' is not defined

In [6]:
p.output(0., np.vstack((results.q_sol[6:], 0.)), np.zeros((1,)))

array([[-0.9],
       [ 0. ]])